# テストの実行

In [31]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

In [32]:
TESTSET_DIR = os.path.join('/home', 'jupyter', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jupyter', 'work')

## 実行するロジックの指定

In [33]:
# 実行プログラムにタグをつけておく
PROG_TAG = 'max_min_euc_dist'

In [34]:
def solve(seed, N, M, D, K, problem, prog_path):
    start_time = time.perf_counter()
    
    command_str = 'echo {} | {}'.format(problem, prog_path)
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    #print('{}'.format(prob_id))    
    return (seed, N, M, D, K, e_time, res)

In [35]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_path = os.path.join(TESTSET_DIR, testset_name+'.csv')
    testset_df = pd.read_csv(testset_path)

    with futures.ThreadPoolExecutor(max_workers=9) as executor:
        for _, row in testset_df.iterrows():
            seed, N, M, D, K, problem = row

            # バッチ実行
            future = executor.submit(solve, seed=seed,  N=N, M=M, D=D, K=K, problem=problem, prog_path=prog_path)
            future_list.append(future)

        _ = futures.as_completed(fs=future_list)

    for future in future_list:
        seed, N, M, D, K, e_time, res = future.result()

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)
        solve_result.append(N)
        solve_result.append(M)
        solve_result.append(D)
        solve_result.append(K)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            # コスト
            cost = int(res.stderr.decode('utf-8').split()[0].replace('Cost=', ''))

            # 非連結なノードペア数
            discon_cnt = int(res.stderr.decode('utf-8').split()[1].replace('DisconCnt=', ''))
                       
        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        solve_result.append(cost)
        solve_result.append(discon_cnt)

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    result_df.columns = ['testset', 'seed', 'N', 'M', 'D', 'K', 'time', 'cost', 'discon_cnt']

    return result_df

In [36]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [37]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            'cost_mean': [int(np.mean(test_result_df['cost']))],
            'cost_min': [min(test_result_df['cost'])],
            'cost_max': [max(test_result_df['cost'])],

            'discon_cnt_mean': [np.mean(test_result_df['discon_cnt'])],
            'discon_cnt_max': [max(test_result_df['discon_cnt'])],

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    summary_all_df = summary_all_df[['tag', 'testset', 'time_mean', 
                                     'cost_mean', 'cost_min', 'cost_max',
                                     'discon_cnt_mean', 'discon_cnt_max',
                                     'time_max']]
    
    return summary_all_df

In [38]:
PROG_NAME_LIST = ['main']
#PROG_NAME_LIST = ['main', 'main_off']

#TEST_LIST = ['00_test_mini']
testset_name = '01_test_pre'
testset_name = '03_test_stress'

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    testset_result_df = run_test(testset_name)
    result_df = pd.concat([result_df, testset_result_df], axis=0)
    
    result_dict[PROG_NAME] = result_df
    summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2023-01-28 23:45:28,236: Start
2023-01-29 00:16:29,965: finish!


In [39]:
result_df

,testset,seed,N,M,D,K,time,cost,discon_cnt
0,03_test_stress,0,791,2277,11,361,2546,10447535,0
1,03_test_stress,1,623,1146,25,70,1133,14323787,0
2,03_test_stress,2,898,1576,8,356,1368,614052174,3584
3,03_test_stress,3,830,1605,12,209,1446,26342149,0
4,03_test_stress,4,700,1500,14,112,1046,16142516,0
...,...,...,...,...,...,...,...,...,...
9995,03_test_stress,9995,879,1701,7,384,1351,50195819,0
9996,03_test_stress,9996,878,1532,15,115,1287,330432751,3504
9997,03_test_stress,9997,731,1537,30,75,1401,98632378,1460
9998,03_test_stress,9998,742,1778,5,614,1013,45966030,0


In [40]:
summary_df = pd.DataFrame()

if testset_name == '03_test_stress' or testset_name == '02_test_sys':
    summary_df = get_summary_df(result_df[:50])
    summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,time_max
0,max_min_euc_dist,01_test_pre,1383,600232315,3754768,14461745934,1280.12,19020,4685


In [41]:
summary_df = pd.DataFrame()

if testset_name == '03_test_stress':
    summary_df = get_summary_df(result_df[:2000])
    summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,time_max
0,max_min_euc_dist,02_test_sys,1614,229058021,3101275,15062345238,889.187,47696,6110


In [42]:
summary_all_dict['main']

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,time_max
0,max_min_euc_dist,03_test_stress,1668,206861415,2935384,19817077682,846.1834,60774,8567


# 結果ログの保存

In [66]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')

for PROG_NAME in PROG_NAME_LIST:
    result_df = result_dict[PROG_NAME]
    
    for testset in np.unique(result_df['testset']):
        csv_df = result_df.query('testset == "{}"'.format(testset))
        csv_df.to_csv(PRJ_DIR+'/result/{}_{}_{}_{}.csv'.format(time_str,PROG_TAG, testset, PROG_NAME), index=False)